In [2]:
import numpy as np
from skfem import (
    MeshTri, Basis, FacetBasis, ElementTriP2, ElementTriP1, ElementVector,
    asm, bmat, condense, solve, BilinearForm, LinearForm, Mesh, ElementTetP2, ElementTetP1
)
from skfem.utils import  solver_iter_krylov
from skfem.models.general import divergence
from skfem.models.poisson import vector_laplace
from skfem.helpers import grad, dot, laplacian
from skfem.helpers import laplacian, precompute_operators, precompute_operators_at_centroids
import matplotlib.pyplot as plt

import scipy.sparse as sp
from scipy.sparse.linalg import eigsh, eigs, splu, LinearOperator



import plotly.graph_objects as go
from skfem.mesh import MeshTet # MeshTet para mallas tetraédricas (3D)

In [3]:
# Mesh
p    = np.linspace(0, 1, 10)
mesh = MeshTet.init_tensor(*(p,) * 3)

# Asignar ID a las fronteras
mesh = mesh.with_boundaries({
    'dirichlet' :  lambda x: (x[0] == 0.0) | (x[0] == 1.0) | 
                             (x[1] == 0.0) | (x[1] == 1.0) | 
                             (x[2] == 0.0) | (x[2] == 1.0),
})

In [4]:
# Definir elementos y bases (P2 para velocidad, P1 para presión) en 3D
element = {
    'u': ElementVector(ElementTetP2(), dim=3),
    'p': ElementTetP1(),
}
basis = {
    'u': Basis(mesh, element['u'], intorder=4),
    'p': Basis(mesh, element['p'], intorder=4),
}
basis_u, basis_p = basis['u'], basis['p']
Nu, Np = basis_u.N, basis_p.N
N      = Nu + Np

In [5]:
@BilinearForm
def mass_matrix(u, v, w):
    return dot(u, v)

# Término convectivo linealizado
@BilinearForm
def convection(u, v, w):
    advection_field = w['w']
    grad_u = grad(u)
    return np.einsum('j...,ij...,i...->...', advection_field, grad_u, v)


# Ensamblaje de matrices
A =  asm(vector_laplace, basis_u)               
B = -asm(divergence, basis_u, basis_p)   

In [44]:
Re_obj = 100
def u_exact_x(x, y, z):
    return y**2
def u_exact_y(x, y, z):
    return -(z**2)
def u_exact_z(x, y, z):
    return x**2
def p_exact(x, y, z):
    return 0*x + 0*y + 0*z + 2

def f_exact_x(x, y, z):
    return -2*y*(z**2) - 2/Re_obj
def f_exact_y(x, y, z):
    return -2*z*(x**2) + 2/Re_obj
def f_exact_z(x, y, z):
    return 2*x*(y**2) - 2/Re_obj

def f_exact_x_stokes(x, y, z):
    return -2 + 0*x
def f_exact_y_stokes(x, y, z):
    return 2 + 0*y
def f_exact_z_stokes(x, y, z):
    return -2 + 0*z

In [45]:
x_boundary = np.zeros(Nu + Np)

# DOFs de las fronteras
dofs_dirichlet  = basis_u.get_dofs('dirichlet').all()
dofs_dirichlet_p  = basis_p.get_dofs('dirichlet').all()

x_ux = basis_u.doflocs[0, dofs_dirichlet[::3]]
y_ux = basis_u.doflocs[1, dofs_dirichlet[::3]]
z_ux = basis_u.doflocs[2, dofs_dirichlet[::3]]


x_boundary[dofs_dirichlet[::3]]  = u_exact_x(x_ux, y_ux, z_ux)  
x_boundary[dofs_dirichlet[1::3]] = u_exact_y(x_ux, y_ux, z_ux)
x_boundary[dofs_dirichlet[2::3]] = u_exact_z(x_ux, y_ux, z_ux)
x_boundary[Nu + dofs_dirichlet_p[0]] = p_exact(basis_p.doflocs[0, dofs_dirichlet_p[0]],
                                               basis_p.doflocs[1, dofs_dirichlet_p[0]],    
                                               basis_p.doflocs[2, dofs_dirichlet_p[0]])

dofs_boundary = np.concatenate([
    dofs_dirichlet,
    np.array([Nu + dofs_dirichlet_p[0]])
])

In [46]:
@LinearForm
def rhs_ns(v, w):
    x, y, z= w.x
    fx = f_exact_x(x, y, z)
    fy = f_exact_y(x, y, z)
    fz = f_exact_z(x, y, z)
    return fx * v[0] + fy * v[1] + fz * v[2]

@LinearForm
def rhs_stokes(v, w):
    x, y, z= w.x
    fx = f_exact_x_stokes(x, y, z)
    fy = f_exact_y_stokes(x, y, z)
    fz = f_exact_z_stokes(x, y, z)
    return fx * v[0] + fy * v[1] + fz * v[2]

# Navier stokes
b_u = asm(rhs_ns, basis_u) 
b_p = np.zeros(Np)  
F_ns   = np.concatenate([b_u, b_p])

# Stokes
b_u_stokes = asm(rhs_stokes, basis_u)
b_p_stokes = np.zeros(Np)
F_stokes   = np.concatenate([b_u_stokes, b_p_stokes])

In [47]:
# Término convectivo linealizado
@BilinearForm
def convection(u, v, w):
    advection_field = w['w']
    grad_u = grad(u)
    return np.einsum('j...,ij...,i...->...', advection_field, grad_u, v)

@BilinearForm
def convection2(u, v, w):
    advection_field = u
    grad_w = grad(w['w'])
    return np.einsum('j...,ij...,i...->...', advection_field, grad_w, v)



In [48]:
def solve_ns_newton(u_init, p_init, Re, max_iter, tol):
    u = u_init
    p = p_init
    for it in range(max_iter):
        # Campo de advección congelado w := u^(it) en puntos de cuadratura
        W = basis_u.interpolate(u)   

        # Ensambla bloque convectivo C(w)
        C2 = asm(convection, basis_u, w=W)

        # Ensambla derivada del bloque convectivo C'(w)
        C1 = asm(convection2, basis_u, w=W)

        # Matriz bloque del paso linealizado
        DF = bmat([[(1/Re) * A + C1 + C2, B.T ],
                  [B,                        None]], format='csr')

        F1 = C2*u + (1/Re)*A*u + B.T*p
        F2 = B*u
        F_real = np.concatenate([F1, F2]) - F_ns

        # Resolver
        delta = solve(*condense(DF, F_real, D=dofs_boundary, x=x_boundary*0))
        u_new = u - delta[:Nu]
        p_new = p - delta[Nu:Nu+Np]

        # Criterio de convergencia
        du = u_new - u
        rel_u = np.linalg.norm(du) / (np.linalg.norm(u_new) + 1e-16)

        dp = p_new - p
        rel_p = np.linalg.norm(dp) / (np.linalg.norm(p_new) + 1e-16)

        # # Sub-relajación si se desea
        u = u_new
        p = p_new

        # print(f"Iteración {it+1}: rel_u = {rel_u:.4e}, rel_p = {rel_p:.4e}")

        if rel_u < tol and rel_p < tol:
            print(f"Convergió en {it+1} iteraciones, residuo {max(rel_u, rel_p):.4e}")
            return u, p, True
    print("No convergió en el número máximo de iteraciones")
    return u, p, False

In [49]:
def solve_ns_picard(u_init, p_init, Re, max_iter, tol):
    u = u_init
    p = p_init
    for it in range(max_iter):
        # Campo de advección congelado w := u^(it) en puntos de cuadratura
        W = basis_u.interpolate(u)   

        # Ensambla bloque convectivo C(w)
        C = asm(convection, basis_u, w=W)

        # Matriz bloque del paso linealizado
        K = bmat([[(1/Re) * A + C, B.T ],
                  [B,              None]], format='csr')

        # Resolver
        sol = solve(*condense(K, F_ns, D=dofs_boundary, x=x_boundary))
        u_new = sol[:Nu]
        p_new = sol[Nu:Nu+Np]

        # Criterio de convergencia
        du = u_new - u
        rel_u = np.linalg.norm(du) / (np.linalg.norm(u_new) + 1e-16)

        dp = p_new - p
        rel_p = np.linalg.norm(dp) / (np.linalg.norm(p_new) + 1e-16)

        # # Sub-relajación si se desea
        u = u_new
        p = p_new

        if rel_u < tol and rel_p < tol:
            print(f"Convergió en {it+1} iteraciones, residuo {max(rel_u, rel_p):.4e}")
            return u, p, True
    print("No convergió en el número máximo de iteraciones")
    return u, p, False

In [50]:
mu = 1.0

# Calcular solución inicial de stokes
K_stokes = bmat([[mu * A,     B.T],  
                 [B,         None]], format='csr')

sol0 = solve(*condense(K_stokes, F_stokes, D=dofs_boundary, x=x_boundary))
u_ref = sol0[:Nu].copy()
p_ref = sol0[Nu:Nu+Np].copy()

In [51]:
# Resolver incrementanto Re
Re = 100
Re_linspace = np.linspace(10, Re, 2)

for R in Re_linspace:
    print(f"Resolviendo para Re = {R:.2f}")
    u_ref, p_ref, flag = solve_ns_picard(u_ref, p_ref, R, max_iter=500, tol=1e-12)

    if not flag:
        print(f"No se pudo converger para este Re = {R:.2f}.")
        break

u_sol = u_ref
p_sol = p_ref


Resolviendo para Re = 10.00
Convergió en 2 iteraciones, residuo 8.8027e-14
Resolviendo para Re = 100.00
Convergió en 2 iteraciones, residuo 1.5351e-13


In [52]:
P1_basis = Basis(mesh, ElementTetP1())

# Mapeo de Presión (Escalar)
pressure_scalar = p_sol 

# Mapeo de Velocidad (Vectorial 3D)
velocity_vector = np.zeros((mesh.nvertices, 3))

for idx in range(mesh.nvertices):
    velocity_vector[idx, 0] = u_sol[basis_u.nodal_dofs[0, idx]]  # componente X
    velocity_vector[idx, 1] = u_sol[basis_u.nodal_dofs[1, idx]]  # componente Y
    velocity_vector[idx, 2] = u_sol[basis_u.nodal_dofs[2, idx]]  # componente Z

point_data_to_export = {
    "velocity": velocity_vector,
    "pressure": pressure_scalar,
}

mesh.save(
        "navierstokes_example3D.vtu", 
        point_data=point_data_to_export,
)

### Strong error

In [53]:
### Error forma fuerte
basis_x = basis_u.split_bases()[0]
basis_y = basis_u.split_bases()[1]
basis_z = basis_u.split_bases()[2]
u_x = u_ref[0::3]
u_y = u_ref[1::3]
u_z = u_ref[2::3] 
p_sol = p_ref 

edofs_x, phix, grad_phix, laplacian_phix = precompute_operators(basis_x)
edofs_y, phiy, grad_phiy, laplacian_phiy = precompute_operators(basis_y)
edofs_z, phiz, grad_phiz, laplacian_phiz = precompute_operators(basis_z)
edofs_p, phip, grad_phip, laplacian_phip = precompute_operators(basis_p)

In [ ]:
# Velocidad y presión en puntos de cuadratura
u_val = np.einsum('dq, de -> qe', phix, u_x[edofs_x]) 
v_val = np.einsum('dq, de -> qe', phiy, u_y[edofs_y])
w_val = np.einsum('dq, de -> qe', phiz, u_z[edofs_z])
p_val = np.einsum('dq, de -> qe', phip, p_sol[edofs_p])

# Calcular operadores en coordenadas físicas
grad_u_fisico_x = np.einsum('de, diqe -> iqe', u_x[edofs_x]  , grad_phix)
grad_u_fisico_y = np.einsum('de, diqe -> iqe', u_y[edofs_y]  , grad_phiy)
grad_u_fisico_z = np.einsum('de, diqe -> iqe', u_z[edofs_z]  , grad_phiz)
grad_p_fisico   = np.einsum('de, diqe -> iqe', p_sol[edofs_p], grad_phip)
laplacian_u_x   = np.einsum('de, de  -> e'   , u_x[edofs_x]  , laplacian_phix)
laplacian_u_y   = np.einsum('de, de   -> e'  , u_y[edofs_y]  , laplacian_phiy)
laplacian_u_z   = np.einsum('de, de   -> e'  , u_z[edofs_z]  , laplacian_phiz)

In [67]:
# Coordenadas de los puntos de cuadratura reales
coords = basis_x.global_coordinates()
X_quad = coords[0]
Y_quad = coords[1]
Z_quad = coords[2]

val_f_x = f_exact_x(X_quad, Y_quad, Z_quad) 
val_f_y = f_exact_y(X_quad, Y_quad, Z_quad) 
val_f_z = f_exact_z(X_quad, Y_quad, Z_quad)

In [69]:
# Residuo momentum en X:

res_momentum_x = np.abs(grad_p_fisico[0] - (laplacian_u_x)/Re + (u_val * grad_u_fisico_x[0] + v_val * grad_u_fisico_x[1] + w_val * grad_u_fisico_x[2]) - val_f_x.T )

# Residuo momentum en Y:
res_momentum_y = np.abs(grad_p_fisico[1] - (laplacian_u_y)/Re + (u_val * grad_u_fisico_y[0] + v_val * grad_u_fisico_y[1] + w_val * grad_u_fisico_y[2]) - val_f_y.T )

# Residuo momentum en Z: 
res_momentum_z = np.abs(grad_p_fisico[2] - (laplacian_u_z)/Re + (u_val * grad_u_fisico_z[0] + v_val * grad_u_fisico_z[1] + w_val * grad_u_fisico_z[2]) - val_f_z.T)

# Residuo Continuidad:
res_continuity = np.abs(grad_u_fisico_x[0] + grad_u_fisico_y[1] + grad_u_fisico_z[2])

print(f"Residuo momentum en X - min: {np.min(res_momentum_x):.4e}, max: {np.max(res_momentum_x):.4e}")
print(f"Residuo momentum en Y - min: {np.min(res_momentum_y):.4e}, max: {np.max(res_momentum_y):.4e}")
print(f"Residuo momentum en Z - min: {np.min(res_momentum_z):.4e}, max: {np.max(res_momentum_z):.4e}")
print(f"Residuo continuidad - min: {np.min(res_continuity):.4e}, max: {np.max(res_continuity):.4e}")


Residuo momentum en X - min: 0.0000e+00, max: 6.5570e-13
Residuo momentum en Y - min: 0.0000e+00, max: 7.0817e-13
Residuo momentum en Z - min: 0.0000e+00, max: 6.8372e-13
Residuo continuidad - min: 0.0000e+00, max: 6.2858e-13
